# Install required packages

In [1]:
!pip install xlsxwriter -q
!pip install keybert -q
!pip install wordwise -q

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from pprint import pprint
import xlsxwriter
from keybert import KeyBERT
from wordwise import Extractor

# Load the Data

In [3]:
# url = "https://www.cloudskillsboost.google/course_templates/72?catalog_rank=%7B%22rank%22%3A1%2C%22num_filters%22%3A0%2C%22has_search%22%3Atrue%7D&search_id=25346376"
# url = "https://www.cloudskillsboost.google/quests/132?catalog_rank=%7B%22rank%22%3A2%2C%22num_filters%22%3A0%2C%22has_search%22%3Atrue%7D&search_id=25346300"
url = "https://www.cloudskillsboost.google/course_templates/53?catalog_rank=%7B%22rank%22%3A1%2C%22num_filters%22%3A0%2C%22has_search%22%3Atrue%7D&search_id=25346338"
page = requests.get(url)
page

<Response [200]>

In [4]:
soup = BeautifulSoup(page.content, "html.parser")
ql_course = soup.find("ql-course")
modules = ql_course.attrs['modules']
print(type(modules))
print("\n\n")
print(modules)

<class 'str'>



[{"id":"59338","title":"Introduction","description":"\u003cp\u003eIn this module, we introduce the course and agenda\u003c/p\u003e","steps":[{"id":"386567","prompt":null,"isOptional":true,"activities":[{"id":"379215","href":null,"isLocked":false,"duration":55000,"title":"Course Introduction","type":"video","isComplete":false,"inProgress":false,"score":null}],"isComplete":false,"allActivitiesRequired":false}],"expanded":false},{"id":"59339","title":"Introduction to Building Batch Data Pipelines","description":"\u003cp\u003eThis module reviews different methods of data loading: EL, ELT and ETL and when to use what\u003c/p\u003e","steps":[{"id":"386568","prompt":null,"isOptional":true,"activities":[{"id":"379216","href":null,"isLocked":false,"duration":69000,"title":"Module introduction","type":"video","isComplete":false,"inProgress":false,"score":null}],"isComplete":false,"allActivitiesRequired":false},{"id":"386569","prompt":null,"isOptional":true,"activities":[{"id":"3

In [5]:
#convert to json
json_modules = json.loads(modules)
print(type(json_modules))
print("\n\n")
pprint(json_modules[0])

<class 'list'>



{'description': '<p>In this module, we introduce the course and agenda</p>',
 'expanded': False,
 'id': '59338',
 'steps': [{'activities': [{'duration': 55000,
                            'href': None,
                            'id': '379215',
                            'inProgress': False,
                            'isComplete': False,
                            'isLocked': False,
                            'score': None,
                            'title': 'Course Introduction',
                            'type': 'video'}],
            'allActivitiesRequired': False,
            'id': '386567',
            'isComplete': False,
            'isOptional': True,
            'prompt': None}],
 'title': 'Introduction'}


In [6]:
df = pd.DataFrame(json_modules)
df

,id,title,description,steps,expanded
0,59338,Introduction,"<p>In this module, we introduce the course and...","[{'id': '386567', 'prompt': None, 'isOptional'...",False
1,59339,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"[{'id': '386568', 'prompt': None, 'isOptional'...",False
2,59340,Executing Spark on Dataproc,<p>This module shows how to run Hadoop on Data...,"[{'id': '386575', 'prompt': None, 'isOptional'...",False
3,59341,Serverless Data Processing with Dataflow,<p>This module covers using Dataflow to build ...,"[{'id': '386587', 'prompt': None, 'isOptional'...",False
4,59342,Manage Data Pipelines with Cloud Data Fusion a...,<p>This module shows how to manage data pipeli...,"[{'id': '386604', 'prompt': None, 'isOptional'...",False
5,59343,Course Summary,<p>Course Summary</p>,"[{'id': '386620', 'prompt': None, 'isOptional'...",False
6,59344,Course Resources,<p>PDF links to all modules</p>,"[{'id': '386621', 'prompt': None, 'isOptional'...",False


In [7]:
flatten_df = pd.json_normalize(json_modules, record_path=['steps', ['activities']], meta=['id','title', 'description'], meta_prefix='meta-', record_prefix='record-')
flatten_df.head(3)

,record-id,record-href,record-isLocked,record-duration,record-title,record-type,record-isComplete,record-inProgress,record-score,meta-id,meta-title,meta-description
0,379215,None,False,55000,Course Introduction,video,False,False,None,59338,Introduction,"<p>In this module, we introduce the course and..."
1,379216,None,False,69000,Module introduction,video,False,False,None,59339,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...
2,379217,None,False,220000,"EL, ELT, ETL",video,False,False,None,59339,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...


In [8]:
flatten_df.drop(['record-href', 'record-isLocked', 'record-isComplete', 'record-inProgress','record-score','meta-id','record-id'], axis=1, inplace=True)

In [9]:
print(flatten_df.shape)
print("\n\n")
print(flatten_df.info())

(58, 5)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   record-duration   58 non-null     int64 
 1   record-title      58 non-null     object
 2   record-type       58 non-null     object
 3   meta-title        58 non-null     object
 4   meta-description  58 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.4+ KB
None


# Aggregating and Grouping the Data

## Using pd.groupby (approach 1)

In [10]:
# grouped_df = flatten_df.groupby(['meta-title', 'meta-description','record-title']) # 58
grouped_df = flatten_df.groupby(['meta-title', 'meta-description']) # 7
print(grouped_df.ngroups)
# print("\n\n")
# print(grouped_df.size())

7


In [11]:
flatten_df['text'] = flatten_df.groupby(['meta-title', 'meta-description'])['record-title'].transform(lambda x:'. '.join(x))
flatten_df.head(10)

,record-duration,record-title,record-type,meta-title,meta-description,text
0,55000,Course Introduction,video,Introduction,"<p>In this module, we introduce the course and...",Course Introduction
1,69000,Module introduction,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
2,220000,"EL, ELT, ETL",video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
3,168000,Quality considerations,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
4,180000,How to carry out operations in BigQuery,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
5,208000,Shortcomings,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
6,428000,ETL to solve data quality issues,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
7,0,Introduction to Building Batch Data Pipelines,quiz,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con..."
8,27000,Module introduction,video,Executing Spark on Dataproc,<p>This module shows how to run Hadoop on Data...,Module introduction. The Hadoop ecosystem. Run...
9,286000,The Hadoop ecosystem,video,Executing Spark on Dataproc,<p>This module shows how to run Hadoop on Data...,Module introduction. The Hadoop ecosystem. Run...


In [12]:
keywords_model = KeyBERT()

In [13]:
def get_keywords_keybert(text, model):
  keywords_arr = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None)
  return [x[0] for x in keywords_arr]

In [14]:
# try wordwise (alternative to keyword extraction)
# def get_keywords_wordwise(text):
#   extractor = Extractor()
#   return extractor.generate(text)

In [15]:
flatten_df['keywords'] = flatten_df['text'].apply(lambda x: get_keywords_keybert(x, keywords_model))
flatten_df

,record-duration,record-title,record-type,meta-title,meta-description,text,keywords
0,55000,Course Introduction,video,Introduction,"<p>In this module, we introduce the course and...",Course Introduction,"[introduction, course]"
1,69000,Module introduction,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
2,220000,"EL, ELT, ETL",video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
3,168000,Quality considerations,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
4,180000,How to carry out operations in BigQuery,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
5,208000,Shortcomings,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
6,428000,ETL to solve data quality issues,video,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
7,0,Introduction to Building Batch Data Pipelines,quiz,Introduction to Building Batch Data Pipelines,<p>This module reviews different methods of da...,"Module introduction. EL, ELT, ETL. Quality con...","[bigquery, pipelines, batch, etl, data]"
8,27000,Module introduction,video,Executing Spark on Dataproc,<p>This module shows how to run Hadoop on Data...,Module introduction. The Hadoop ecosystem. Run...,"[hadoop, dataproc, hdfs, spark, cloud]"
9,286000,The Hadoop ecosystem,video,Executing Spark on Dataproc,<p>This module shows how to run Hadoop on Data...,Module introduction. The Hadoop ecosystem. Run...,"[hadoop, dataproc, hdfs, spark, cloud]"


## Using set_index (approach 2)

In [16]:
index_df = flatten_df.set_index(['meta-title', 'meta-description','record-title'])

In [17]:
print(f'index names ----> {index_df.index.names}')
print('\n')
print(f'column names ----> {index_df.columns}')

index names ----> ['meta-title', 'meta-description', 'record-title']


column names ----> Index(['record-duration', 'record-type', 'text', 'keywords'], dtype='object')


In [18]:
index_df.index.names = ['titles', 'description', 'activities']
index_df.rename(columns={'record-duration':'duration', 'record-type':'type'}, errors='raise')

duration  \
titles                                             description                                        activities                                                     
Introduction                                       <p>In this module, we introduce the course and ... Course Introduction                                    55000   
Introduction to Building Batch Data Pipelines      <p>This module reviews different methods of dat... Module introduction                                    69000   
                                                                                                      EL, ELT, ETL                                          220000   
                                                                                                      Quality considerations                                168000   
                                                                                                      How to carry out operations in BigQuery               180000   
                                                                                                      Shortcomings                                          208000   
                                                                                                      ETL to solve data quality issues                      428000   
                                                                                                      Introduction to Building Batch Data Pipelines              0   
Executing Spark on Dataproc                        <p>This module shows how to run Hadoop on Datap... Module introduction                                    27000   
                                                                                                      The Hadoop ecosystem                                  286000   
                                                                                                      Running Hadoop on Dataproc                            602000   
                                                                                                      Cloud Storage instead of HDFS                         379000   
                                                                                                      Optimizing Dataproc                                   172000   
                                                                                                      Optimizing Dataproc storage                           567000   
                                                                                                      Optimizing Dataproc templates and autoscaling         312000   
                                                                                                      Optimizing Dataproc monitoring                        186000   
                                                                                                      Lab Intro: Running Apache Spark jobs on Dataproc       28000   
                                                                                                      Running Apache Spark jobs on Cloud Dataproc          5400000   
                                                                                                      Summary                                                34000   
                                                                                                      Executing Spark on Dataproc                                0   
Serverless Data Processing with Dataflow           <p>This module covers using Dataflow to build y... Module introduction                                    53000   
                                                                                                      Introduction to Dataflow                              342000   
                                                                                                      Why customers value Dataflow                          171000   
    

# Save output to Excel workbook

In [19]:
with pd.ExcelWriter('demo_GCBS.xlsx', engine='xlsxwriter') as writer:
  # flatten_df.to_excel(writer, sheet_name="CLMG004")
  index_df.to_excel(writer, sheet_name="CLMG004")